(PERFBLAS)=

# 5.2 Herramientas de lenguajes de programación y del sistema operativo para perfilamiento e implementaciones de BLAS

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* Aprenderá diferentes herramientas para perfilamiento de código y dará interpretación a los diferentes resultados.

* Conocerá algunas operaciones de BLAS en los tres niveles. En específico se utilizará *OpenBLAS* para *NumPy* y *R*.

```

En esta nota revisamos algunas herramientas de los lenguajes de programación y de los sistemas GNU/Linux para perfilamiento de código. También se revisa la implementación *OpenBLAS* de la especificación de *Basic Linear Algebra Subprograms* (BLAS) en sus diferentes niveles con ejemplos de paquetes en los lenguajes.

```{margin}

Utilizamos la máquina `m4.16xlarge` pues en AWS no todas las instancias se pueden obtener algunas métricas con `perf`. Ver [stackoverflow1](https://stackoverflow.com/questions/45683901/event-cache-misses-not-supported-by-perf-in-aws), [stackoverflow2](https://stackoverflow.com/questions/19763070/ubuntu-12-10-perf-stat-not-supported-cycles), [the-pmcs-of-ec2](http://www.brendangregg.com/blog/2017-05-04/the-pmcs-of-ec2.html)

```

Se presentan códigos y sus ejecuciones en una máquina `m4.16xlarge` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2290.913
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-140 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con *import*'s. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

```{admonition} Comentario

Lo más recomendable para realizar perfilamiento de códigos es realizarlos **no** en *jupyter notebooks*. A continuación se realizarán algunas mediciones en este *notebook* para propósitos didácticos pero se sugiere perfilar en el intérprete de *Python* o ejecutando un `script.py`.

```

## Perfilamiento: medición de tiempo en *Python* y *IPython*

Lo más natural que podemos pensar en medir es el tiempo de ejecución de nuestros códigos. *Python* y *IPython* tienen herramientas para este propósito.

### Módulo: [time](https://docs.python.org/3/library/time.html#time.time)

Imports y funciones de apoyo:

In [4]:
import math
import time

import numpy as np
from pytest import approx
from scipy.integrate import quad

In [5]:
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    nodes = [a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res = 0
    for node in nodes:
        sum_res = sum_res+f(node)
    return h_hat*sum_res


Objetivo:

In [6]:
f = lambda x: math.exp(-x**2) #using math library
a = 0
b = 1
obj, err = quad(f, a, b)
print(obj)

0.7468241328124271


Medición de tiempo:

In [7]:
n = 10**6
start_time = time.time()
res = Rcf(f,a,b,n)
end_time = time.time()
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.39637255668640137 segundos


Prueba que se resuelve correctamente el problema:

In [8]:
print(res == approx(obj))

True


### Comando de *magic*: [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

Nos regresa las mediciones siguientes:

* `CPU times` que contiene:

    * `user`: mide la cantidad de tiempo de los *statements* que la CPU gastó para funciones que no están relacionadas con el *kernel* del sistema.
    
    * `sys`: mide la cantidad de tiempo de los *statements* que la CPU gastó en funciones a nivel de kernel del sistema.
    
    * `total`: suma entre el `user` y `sys` para todos **todos los *cores***. 
    
* `Wall time`: mide el *wall clock* o *elapsed time* que se refiere al tiempo desde que inicia la ejecución de los *statements* hasta su finalización.

* `Out`: resultado.



In [9]:
%time Rcf(f,a, b,n)

CPU times: user 365 ms, sys: 20.4 ms, total: 386 ms
Wall time: 385 ms


0.7468241328124773

```{admonition} Comentarios

* Para mediciones de tiempos que involucran cómputo en paralelo es posible que `total` exceda a `Wall time` por una diferencia notable pues es la suma de tiempos para todos los *cores*.

* Recuérdese que algunos ejemplos de funciones relacionadas con el *kernel* del sistema es el alojamiento, lectura y escritura de variables en memoria, las relacionadas con el I/O de disco o *network*.

* La diferencia entre `total` y `Wall time` da una idea de la cantidad de tiempo que se ocupó el sistema en la ejecución de *statements* que no involucran `user` ni a `sys` (por ejemplo un *statement* del tipo `sleep`) o en tareas no relacionadas con los *statements* medidos (por ejemplo si en el momento de la medición se estaban corriendo otros procesos).

```

Al medir tiempos de ejecución, **siempre** hay variación en la medición por lo que se recomienda realizar las mediciones un número repetido de veces. Para este caso *Timeit* nos ayuda. 

### [Timeit](https://docs.python.org/3/library/timeit.html)

Se ejecuta desde la línea de comandos, con el comando de *magic* [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) o realizando `import`.

A continuación se mide el tiempo de ejecución para la función `Rcf`. Se promedian los tiempos de $n=5$ ejecuciones y se calcula su desviación estándar. Se reptite lo anterior $r=10$ veces y se reporta el mejor resultado. 

In [10]:
%timeit -n 5 -r 10 Rcf(f,a, b,n)

373 ms ± 1.91 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


```{admonition} Observación
:class: tip

$ms$ es milisecond, $\mu s$ es microsecond y $ns$ es nanosecond.

```

```{admonition} Comentarios

* `%timeit` se recomienda usar para secciones de código pequeñas. Si se observa una variación en las mediciones de tiempo entre distintas repeticiones entonces hay que realizar más repeticiones hasta tener un resultado estable.

* `%timeit` desabilita temporalmente el *garbage collector* de *Python* (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Si el *garbage collector* es invocado en tus *statements*, esto puede ser una razón de posibles diferencias que se obtengan en las mediciones de tiempo.

* Para su uso con un `import` ver [timeit](https://docs.python.org/3/library/timeit.html) y [how-to-use-timeit-module](https://stackoverflow.com/questions/8220801/how-to-use-timeit-module).


```

```{admonition} Ejercicio
:class: tip

Implementar la regla de Simpson compuesta para aproximar la misma integral y medir los tiempos de ejecución con un mismo número de subintervalos. Revisar que se está aproximando correctamente el valor numérico con `quad`.

```

## Perfilamiento: medición de uso de CPU en *Python*

### [cProfile](https://docs.python.org/3/library/profile.html) 

*cProfile* está en la *standard-library* de Python como *built-in*. Se utiliza con la implementación *CPython* de *Python* para medir el tiempo de ejecución de cada función en el programa. Se ejecuta desde la línea de comandos, con un comando de *magic* o realizando `import`.

In [11]:
import cProfile

In [12]:
n=10**6
cprof = cProfile.Profile()
cprof.enable()
res = Rcf(f,a,b,n)
cprof.disable()
cprof.print_stats(sort='cumtime')

         2000047 function calls in 3.007 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.006    1.503 interactiveshell.py:3396(run_code)
        2    0.000    0.000    3.006    1.503 {built-in method builtins.exec}
        1    0.020    0.020    3.006    3.006 <ipython-input-12-ad50f78947e3>:4(<module>)
        1    0.749    0.749    2.986    2.986 <ipython-input-5-303d3a3bb088>:1(Rcf)
  1000000    1.443    0.000    2.101    0.000 <ipython-input-6-e098c9ed7b86>:1(<lambda>)
  1000000    0.658    0.000    0.658    0.000 {built-in method math.exp}
        1    0.136    0.136    0.136    0.136 <ipython-input-5-303d3a3bb088>:23(<listcomp>)
        2    0.000    0.000    0.000    0.000 codeop.py:140(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 contextlib.py:238(helper)
        2    0.000    0.000    0.000

El *output* de *cProfile* muestra:

* El tiempo total de ejecución, el cual incluye el tiempo del bloque de código que estamos midiendo y el overhead al usar *cProfile*. Por esta razón se tiene un mayor tiempo de ejecución que con las mediciones de tiempo anteriores.

* La columna `ncalls` que como el nombre indica, muestra el número de veces que se llamó a cada función. En este caso las funciones `lambda` y `math.exp` son las que se llaman un mayor número de veces: $n=10^6$ veces. 

* La columna `tottime` muestra el tiempo que tardaron estas funciones en ejecutarse (sin llamar a otras funciones).

* La columna `percall` es el cociente entre `tottime` y `ncalls`.

* La columna `cumtime` contiene el tiempo gastado en la función y en las demás que llama. Por ejemplo la función `Rcf` llama a `listcomp` por lo que es natural que `Rcf` esté más arriba en el *output* ordenado de *cProfile*. Esto también ocurre con `lambda` y `math.exp` pues la primera llama a la segunda.

* La columna de `percall` es un cociente entre la columna `cumtime` y el conteo del número de veces que se llamaron a funciones primitivas o también nombradas *built in functions*.

* La última columna indica información de la función y la línea en la que se encuentra dentro del código. Por ejemplo para el caso de `listcomp` sale la línea 23 y si se va a la celda en donde se definió `Rcf` corresponde tal línea al llamado `nodes=[a+(i+1/2)*h_hat for i in range(0,n)]`. 


````{admonition} Comentarios

* El comando de *magic* para ejecutar *cProfile* en una celda del *jupyter notebook* es: `%prun -s cumulative Rcf(f,a, b,n)`

* *cProfile* puede ejecutarse desde la línea de comando con: `python3 -m cProfile -s cumulative Rcf.py` creando `Rcf.py` por ejemplo con:


```python
%%file Rcf.py
import math

from pytest import approx
from scipy.integrate import quad

def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

if __name__ == "__main__":
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))
```

La flag `-s` con la ejecución vía línea de comando, indica que se ordene el resultado por el tiempo acumulado dentro de cada función.


````

El *output* de *cProfile* no es muy flexible por lo que tenemos el módulo `pstats` con la clase `Stats` para manipular los resultados dentro de *Python*. Por ejemplo para ordenar por *cumulative time* e imprimir sólo las 10 líneas más significativas se puede realizar:

```{margin}

Creamos el archivo `Rcf_stats` para ser usado por `pstats`.

```

In [13]:
cprof.dump_stats("Rcf_stats")

In [14]:
import pstats

In [15]:
p_rcf_stats = pstats.Stats("Rcf_stats")
print(p_rcf_stats.sort_stats("cumulative").print_stats(10))

Tue Apr 13 02:10:42 2021    Rcf_stats

         2000047 function calls in 3.007 seconds

   Ordered by: cumulative time
   List reduced from 25 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.006    1.503 /home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3396(run_code)
        2    0.000    0.000    3.006    1.503 {built-in method builtins.exec}
        1    0.020    0.020    3.006    3.006 <ipython-input-12-ad50f78947e3>:4(<module>)
        1    0.749    0.749    2.986    2.986 <ipython-input-5-303d3a3bb088>:1(Rcf)
  1000000    1.443    0.000    2.101    0.000 <ipython-input-6-e098c9ed7b86>:1(<lambda>)
  1000000    0.658    0.000    0.658    0.000 {built-in method math.exp}
        1    0.136    0.136    0.136    0.136 <ipython-input-5-303d3a3bb088>:23(<listcomp>)
        2    0.000    0.000    0.000    0.000 /usr/lib/python3.8/codeop.py:140(__call__)
        2    0.0

Si queremos filtrar los resultados para los que hagan *match* con las palabras `lambda` o `listcomp` o `math` tomadas como expresiones regulares en la columna `filename:lineno(function)`:

In [16]:
print(p_rcf_stats.sort_stats("cumulative").print_stats("lambda|listcomp|math"))

Tue Apr 13 02:10:42 2021    Rcf_stats

         2000047 function calls in 3.007 seconds

   Ordered by: cumulative time
   List reduced from 25 to 3 due to restriction <'lambda|listcomp|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000000    1.443    0.000    2.101    0.000 <ipython-input-6-e098c9ed7b86>:1(<lambda>)
  1000000    0.658    0.000    0.658    0.000 {built-in method math.exp}
        1    0.136    0.136    0.136    0.136 <ipython-input-5-303d3a3bb088>:23(<listcomp>)




Y podemos usar `strip_dirs` para eliminar el nombre de la ruta para mayor legibilidad:

In [17]:
print(p_rcf_stats.strip_dirs().sort_stats("cumulative").print_stats("lambda|listcomp|math"))

Tue Apr 13 02:10:42 2021    Rcf_stats

         2000047 function calls in 3.007 seconds

   Ordered by: cumulative time
   List reduced from 25 to 3 due to restriction <'lambda|listcomp|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000000    1.443    0.000    2.101    0.000 <ipython-input-6-e098c9ed7b86>:1(<lambda>)
  1000000    0.658    0.000    0.658    0.000 {built-in method math.exp}
        1    0.136    0.136    0.136    0.136 <ipython-input-5-303d3a3bb088>:23(<listcomp>)




El número de veces que se llamaron a funciones primitivas o *built in* es:

In [18]:
print(p_rcf_stats.prim_calls)

2000047


También `pstats` nos permite obtener qué funciones llaman a otras o son llamadas:

In [19]:
p_rcf_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
interactiveshell.py:3396(run_code)                <- 
{built-in method builtins.exec}                   <-       2    0.000    3.006  interactiveshell.py:3396(run_code)
<ipython-input-12-ad50f78947e3>:4(<module>)       <-       1    0.020    3.006  {built-in method builtins.exec}
<ipython-input-5-303d3a3bb088>:1(Rcf)             <-       1    0.749    2.986  <ipython-input-12-ad50f78947e3>:4(<module>)
<ipython-input-6-e098c9ed7b86>:1(<lambda>)        <- 1000000    1.443    2.101  <ipython-input-5-303d3a3bb088>:1(Rcf)
{built-in method math.exp}                        <- 1000000    0.658    0.658  <ipython-input-6-e098c9ed7b86>:1(<lambda>)
<ipython-input-5-303d3a3bb088>:23(<listcomp>)     <-       1    0.136    0.136  <ipython-input-5-303d3a3bb088>:1(Rcf)
codeop.py:140(__call__)                           <- 
{bui

In [20]:
p_rcf_stats.strip_dirs().sort_stats("cumulative").print_callees()

   Ordered by: cumulative time

Function                                          called...
                                                      ncalls  tottime  cumtime
interactiveshell.py:3396(run_code)                ->       2    0.000    0.000  hooks.py:103(__call__)
                                                           2    0.000    0.000  interactiveshell.py:1278(user_global_ns)
                                                           2    0.000    0.000  ipstruct.py:125(__getattr__)
                                                           2    0.000    3.006  {built-in method builtins.exec}
{built-in method builtins.exec}                   ->       1    0.020    3.006  <ipython-input-12-ad50f78947e3>:4(<module>)
                                                           1    0.000    0.000  <ipython-input-12-ad50f78947e3>:5(<module>)
<ipython-input-12-ad50f78947e3>:4(<module>)       ->       1    0.749    2.986  <ipython-input-5-303d3a3bb088>:1(Rcf)
<ipython-input-5-3

Análogamente podemos establecer los filtros usados en `print_stats`:

In [21]:
p_rcf_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 25 to 10 due to restriction <10>

Function                                       was called by...
                                                   ncalls  tottime  cumtime
interactiveshell.py:3396(run_code)             <- 
{built-in method builtins.exec}                <-       2    0.000    3.006  interactiveshell.py:3396(run_code)
<ipython-input-12-ad50f78947e3>:4(<module>)    <-       1    0.020    3.006  {built-in method builtins.exec}
<ipython-input-5-303d3a3bb088>:1(Rcf)          <-       1    0.749    2.986  <ipython-input-12-ad50f78947e3>:4(<module>)
<ipython-input-6-e098c9ed7b86>:1(<lambda>)     <- 1000000    1.443    2.101  <ipython-input-5-303d3a3bb088>:1(Rcf)
{built-in method math.exp}                     <- 1000000    0.658    0.658  <ipython-input-6-e098c9ed7b86>:1(<lambda>)
<ipython-input-5-303d3a3bb088>:23(<listcomp>)  <-       1    0.136    0.136  <ipython-input-5-303d3a3bb088>:1(Rcf)
codeop.py:140(__call__)        

In [22]:
p_rcf_stats.strip_dirs().sort_stats("cumulative").print_callees("Rcf|lambda")

   Ordered by: cumulative time
   List reduced from 25 to 2 due to restriction <'Rcf|lambda'>

Function                                    called...
                                                ncalls  tottime  cumtime
<ipython-input-5-303d3a3bb088>:1(Rcf)       ->       1    0.136    0.136  <ipython-input-5-303d3a3bb088>:23(<listcomp>)
                                               1000000    1.443    2.101  <ipython-input-6-e098c9ed7b86>:1(<lambda>)
<ipython-input-6-e098c9ed7b86>:1(<lambda>)  -> 1000000    0.658    0.658  {built-in method math.exp}




```{admonition} Comentario

*cProfile* sólo mide tiempos para llamadas a funciones de forma explícita no métodos que son llamados de forma interna como en el siguiente ejemplo en el que se hace una extracción de la primer columna de un *NumPy* *array*.

```

In [23]:
def myfunc():
    A = np.array([[1, 2, 0],
                  [3, 4, 5]], dtype=float)
    a_1 = A[:, 0]
    return a_1

In [24]:
%prun -s cumulative myfunc()

         5 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-23-665771079aea>:1(myfunc)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

```{admonition} Comentario

*cProfile* puede usarse en compañía del paquete [SnakeViz](https://jiffyclub.github.io/snakeviz/) para visualizar los resultados.

```

```{admonition} Ejercicio
:class: tip

Perfilar la regla de Simpson que se implementó en el primer ejercicio con `cProfile` utilizando su `import` y  el paquete `pstats`.


```

### [line_profiler](https://github.com/pyutils/line_profilerhttps://github.com/pyutils/line_profiler)

Nos ayuda a perfilar el código línea por línea. 

Una buena idea es perfilar el código primero con *cProfile* para identificar aquellas funciones que gastan un mayor tiempo de ejecución y posteriormente perfilarlas con *line_profiler*.

Se ejecuta desde la línea de comandos, con un comando de magic o realizando `import`.

In [25]:
import line_profiler

In [26]:
line_prof = line_profiler.LineProfiler()
n = 10**6
print(line_prof(Rcf)(f,a,b,n))

0.7468241328124773


In [27]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 2.9941 s
File: <ipython-input-5-303d3a3bb088>
Function: Rcf at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point
     4                                               method in an interval.
     5                                               Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
     6                                               i=0,1,...,n-1 and h_hat=(b-a)/n
     7                                               Args:
     8                                               
     9                                                   f (function): function expression of integrand.
    10                                                   
    11                                         

El *output* de *line_profiler* muestra:

* *Line* #: el número de la línea.

* *Hits*: el número de veces que la línea fue ejecutada.

* *Time* el tiempo que tomó la ejecución de la línea. Las unidades en el que se mide el tiempo está en la primer línea: `Timer unit`.

* *Per Hit*: el tiempo promedio que toma la ejecución de la línea una vez en `timer's units`.

* *\% Time*: el porcentaje de tiempo que toma la ejecución de la línea correspondiente al tiempo total de ejecución de la función.

* *Line contents*: el código a perfilar.

```{admonition} Comentarios

* Para cambiar `Timer unit` puede usarse: `line_prof.print_stats(output_unit=1e-03)`.

* Con `add_function` de *line_profiler* es posible perfilar varias funciones.

```

Utilizando *generators*:

In [28]:
def Rcf_2(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


In [29]:
line_prof.add_function(Rcf_2)
print(line_prof(Rcf_2)(f, a, b, n))

0.7468241328124773


Utilizando funcionalidad de *NumPy*:

In [30]:
f_np = lambda x: np.exp(-x**2)

In [31]:
def Rcf_3(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    aux_vec = np.linspace(a, b, n+1)
    nodes= (aux_vec[:-1]+aux_vec[1:])/2
    return h_hat*np.sum(f(nodes))


In [32]:
line_prof.add_function(Rcf_3)
print(line_prof(Rcf_3)(f_np, a, b, n))

0.7468241328124576


In [35]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 3.15798 s
File: <ipython-input-28-f9354d0b879d>
Function: Rcf_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_2(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point
     4                                               method in an interval.
     5                                               Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
     6                                               i=0,1,...,n-1 and h_hat=(b-a)/n
     7                                               Args:
     8                                               
     9                                                   f (function): function expression of integrand.
    10                                                   
    11                                   

```{admonition} Comentarios

* Para su uso con el comando de *magic* hay que cargarlo como una extensión de *jupyter*: `%load_ext line_profiler` y posteriormente usar: `%lprun -f Rcf Rcf(f, a, b, n)`.

* Para su uso vía la línea de comandos se debe ejecutar: `kernprof -v -l <script.py>` o bien `python3 <script.py>` como se muestra a continuación. La opción `-v` es *view the results*, `-l` indica que se haga el perfilamiento línea a línea. Por *default* se escribe un archivo con extensión `.lprof` el cual puede revisarse con `python3 -m line_profiler <script.py.lprof>`

```

In [36]:
%%file Rcf_line_profiler.py
import math

from pytest import approx
from scipy.integrate import quad

@profile #decorate the functions you want to profile with kernprof 
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

if __name__ == "__main__":
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))

Writing Rcf_line_profiler.py


In [37]:
%%bash
$HOME/.local/bin/kernprof -l -v Rcf_line_profiler.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True
Wrote profile results to Rcf_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 3.03486 s
File: Rcf_line_profiler.py
Function: Rcf at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           @profile #decorate the functions you want to profile with kernprof 
     7                                           def Rcf(f,a,b,n):
     8                                               """
     9                                               Compute numerical approximation using rectangle or mid-point
    10                                               method in an interval.
    11                                               Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    12                                               i=0,1,...,n-1 and h_hat=(b-a)/n
    13                                               Args:
    14                                           

Utilizando `python3 <script.py>`:

In [39]:
%%file Rcf_line_profiler_2.py
import math

import line_profiler
from pytest import approx
from scipy.integrate import quad

def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

if __name__ == "__main__":
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))
    line_prof = line_profiler.LineProfiler()
    print(line_prof(Rcf)(f,a,b,n))
    line_prof.print_stats()    

Overwriting Rcf_line_profiler_2.py


In [40]:
%%bash
python3 Rcf_line_profiler_2.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True
0.7468241328124773
Timer unit: 1e-06 s

Total time: 3.12149 s
File: Rcf_line_profiler_2.py
Function: Rcf at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def Rcf(f,a,b,n):
     8                                               """
     9                                               Compute numerical approximation using rectangle or mid-point
    10                                               method in an interval.
    11                                               Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    12                                               i=0,1,...,n-1 and h_hat=(b-a)/n
    13                                               Args:
    14                                               
    15                                                   f (function): function expression of integrand.
    16                                

```{admonition} Ejercicio
:class: tip

Perfilar la regla de Simpson que se implementó en el primer ejercicio con `line_profiler` utilizando su `import`.


```

## Perfilamiento: medición de uso de memoria en *Python*

Si bien las computadoras de hoy en día tienen una gran cantidad de RAM es importante que las aplicaciones no utilicen la totalidad pues en ese caso se tendrá una penalización en el *performance* de la aplicación al utilizar [virtual memory](https://en.wikipedia.org/wiki/Virtual_memory), ver [espacio_de_intercambio](https://es.wikipedia.org/wiki/Espacio_de_intercambio).

### [memory_profiler](https://github.com/pythonprofilers/memory_profiler)

Se ejecuta desde la línea de comandos, con un comando de *magic* o realizando `import`. Al instalar `memory_profiler` se incluyen dos comandos de *magic*: `%memit` y `%mprun`. `%mprun` es similar a *line_profiler* al analizar línea por línea el uso de memoria.

En el caso de `import` regresa una lista de valores de uso de memoria en *MiB* medidas cada cierto `interval` (argumento de `memory_usage`). En lo siguiente se pide que se regrese el máximo uso de memoria de la lista.

In [41]:
from memory_profiler import memory_usage

In [42]:
t = (Rcf, (f, a, b, n))
print(memory_usage(t, max_usage=True))

123.69140625


Y podemos ejecutar las siguientes líneas de acuerdo a [how-do-i-profile-memory-usage-in-python](https://stackoverflow.com/questions/552744/how-do-i-profile-memory-usage-in-python) para consistencia con los resultados que se obtendrán con `%memit` y `%mprun`:

In [43]:
start_mem = memory_usage(max_usage=True)
res = memory_usage(t, max_usage=True, retval=True)
print('start mem', start_mem)
print('max mem', res[0])
print('used mem', res[0]-start_mem)
print('fun output', res[1])

start mem 93.02734375
max mem 123.6171875
used mem 30.58984375
fun output 0.7468241328124773


Para el uso de `%memit` y `%mprun` se carga la extensión de *jupyter*:

In [44]:
%load_ext memory_profiler

`%memit` devuelve el pico de memoria usada en una celda de un *jupyter notebook* y utiliza las mismas ideas para reportar las mediciones que `%timeit`:

In [45]:
%memit #how much RAM this process is consuming

peak memory: 93.30 MiB, increment: 0.00 MiB


In [46]:
%memit -c Rcf(f, a, b, n)

peak memory: 194.29 MiB, increment: 100.99 MiB


In [47]:
%memit -c Rcf_2(f, a, b, n)

peak memory: 164.54 MiB, increment: 70.95 MiB


In [48]:
%memit -c Rcf_3(f_np, a, b, n)

peak memory: 172.20 MiB, increment: 78.62 MiB


```{admonition} Comentario

Si se utiliza `%mprun` debe crearse `script.py` y utilizar un *decorator*.

```

In [50]:
%%file Rcf_memory_profiler.py

import math

from pytest import approx
from scipy.integrate import quad
from memory_profiler import profile

@profile #decorate the functions you want to profile with memory_profiler 
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

if __name__ == "__main__":
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))
    

Overwriting Rcf_memory_profiler.py


In [51]:
from Rcf_memory_profiler import Rcf_mprun

In [52]:
%mprun -f Rcf_mprun Rcf_mprun(f, a, b, n)

Filename: /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.2/Rcf_memory_profiler.py

Line #    Mem usage    Increment  Occurences   Line Contents
     8     86.1 MiB     86.1 MiB           1   @profile #decorate the functions you want to profile with memory_profiler 
     9                                         def Rcf_mprun(f,a,b,n):
    10                                             """
    11                                             Compute numerical approximation using rectangle or mid-point
    12                                             method in an interval.
    13                                             Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    14                                             i=0,1,...,n-1 and h_hat=(b-a)/n
    15                                             Args:
    16                                             
    17                                                 f (function

Filename: /home/ubuntu/.local/lib/python3.8/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurences   Line Contents
  1140     86.1 MiB     86.1 MiB           1               def wrapper(*args, **kwargs):
  1141     86.1 MiB      0.0 MiB           1                   prof = get_prof()
  1142     93.8 MiB      7.7 MiB           1                   val = prof(func)(*args, **kwargs)
  1143     93.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1144     93.8 MiB      0.0 MiB           1                   return val

El *output* de `%mprun` contiene:

* La primera columna hace referencia a la línea de código.

* *Mem usage* es la memoria usada por *Python* después que tal línea ha sido ejecutada. En el caso anterior se tienen de inicio una cantidad positiva de `MiB` para cargar el script `script.py`, realizar `imports`.

* *Increment* representa la diferencia en memoria de la línea actual respecto a la línea anterior. Las cantidades que se muestran en esta columna no necesariamente indican que en memoria se alojan tales cantidades.

* *Ocurrences* contador del número de veces que se ejecuta tal línea.

* *Line contents* el código a perfilar.


Ejemplo ejecutado desde la línea de comandos:

In [53]:
%%bash
python3 Rcf_memory_profiler.py

objetivo: 7.468241e-01
Filename: Rcf_memory_profiler.py

Line #    Mem usage    Increment  Occurences   Line Contents
     8     74.8 MiB     74.8 MiB           1   @profile #decorate the functions you want to profile with memory_profiler 
     9                                         def Rcf_mprun(f,a,b,n):
    10                                             """
    11                                             Compute numerical approximation using rectangle or mid-point
    12                                             method in an interval.
    13                                             Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    14                                             i=0,1,...,n-1 and h_hat=(b-a)/n
    15                                             Args:
    16                                             
    17                                                 f (function): function expression of integrand.
    18                                      

```{admonition} Comentarios

* De [FAQ](https://github.com/pythonprofilers/memory_profiler#frequently-asked-questions)

*Q: How accurate are the results ?
A: This module gets the memory consumption by querying the operating system kernel about the amount of memory the current process has allocated, which might be slightly different from the amount of memory that is actually used by the Python interpreter. Also, because of how the garbage collector works in Python the result might be different between platforms and even between runs.*

* También *memory_profiler* contiene un *script* de nombre `mprof` que permite monitorear uso de memoria de un proceso (que puede ser un *Python script* u otro) y también soporta el monitoreo de *child processes* que fueron *spawn* de un proceso en un contexto de [multiprocessing-contexts-and-start-methods](https://docs.python.org/3.9/library/multiprocessing.html#contexts-and-start-methods), ver [memory_profiler#time-based-memory-usage](https://github.com/pythonprofilers/memory_profiler#time-based-memory-usage)


```

```{admonition} Ejercicio
:class: tip

Perfilar la regla de Simpson que se implementó en el primer ejercicio con `memory_profiler` utilizando su `import`.


```

### [heapy](https://github.com/zhuyifei1999/guppy3/)

*Python* utiliza un *memory manager* para la administración de la memoria dinámica *heap*. Aunque *Python* maneja la memoria automáticamente pueden darse situaciones en las que referencias a objetos ya no sean necesarias o bien tener [memory leaks](https://en.wikipedia.org/wiki/Memory_leak). En tal caso podría llamarse al [garbage collector](https://docs.python.org/3/library/gc.html) (administrado por el *memory manager*) de forma manual y usar la herramienta de *heapy* para monitorear la memoria *heap*.

```{margin}

Ver [stackoverflow: What is a Memory Heap?](https://stackoverflow.com/questions/2308751/what-is-a-memory-heap), [Python: memory management](https://docs.python.org/3/c-api/memory.html), [stackoverflow: does-python-have-a-stack-heap-and-how-is-memory-managed](https://stackoverflow.com/questions/14546178/does-python-have-a-stack-heap-and-how-is-memory-managed).

```

La herramienta *heapy* nos ayuda a obtener información acerca de los objetos en la memoria *heap*. Por ejemplo podemos revisar el número y tamaño de cada objeto que está en la *heap*. También nos ayuda a encontrar *memory leaks*.

*heapy* funciona escribiendo líneas para llamar a la función `heap` que encierren las secciones de código que queremos monitorear. Subsecuentes llamados a tal función nos proporciona información en una forma incremental.

In [54]:
%%file heapy_ex_1
import math

from guppy import hpy

a=0
b=1
n=10**6
f=lambda x: math.exp(-x**2)
hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
h_hat = (b-a)/n
h = hp.heap()
print(h)

Writing heapy_ex_1


In [55]:
%%bash
python3 heapy_ex_1

Partition of a set of 2 objects. Total size = 432 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  50      408  94       408  94 types.FrameType
     1      1  50       24   6       432 100 float


El *output* anterior muestra:

* *Partition of a set of ... objects* que da el número de objetos analizados y clasificados de acuerdo al `Kind`.

* *Total size* tamaño de la *heap* por cada llamado a la función `heap` del monitoreo. Ayuda a las comparaciones del monitoreo subsecuentes.

* *Index* para enumerar cada objeto en la *heap*. Se puede utilizar para obtener información utilizando índices, por ejemplo `h[0]` o `h[1]`:

In [56]:
%%file heapy_ex_1_b
import math

from guppy import hpy

hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
a=0
b=1
n=10**6
f=lambda x: math.exp(-x**2)
h_hat = (b-a)/n
h = hp.heap()
print(h)
print("index 1:")
print(h[1])

Writing heapy_ex_1_b


In [57]:
%%bash
python3 heapy_ex_1_b

Partition of a set of 3 objects. Total size = 568 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  33      408  72       408  72 types.FrameType
     1      1  33      136  24       544  96 function
     2      1  33       24   4       568 100 float
index 1:
Partition of a set of 1 object. Total size = 136 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     1      1 100      136 100       544 400 function


* *Count* un contador del número de objetos en el reporte de acuerdo al *Kind*. En seguida se muestra su porcentaje de acuerdo al número de objetos analizados.

* *Size* el tamaño que ocupa en la *heap* de acuerdo al *Total size*. En seguida se muestra su porcentaje correspondiente.

* *Cumulative* muestra por cada objeto analizado de acuerdo al *Kind* de lo que acumulan en la *heap*. El último renglón del reporte equivale al *Total size*. En seguida se muestra su porcentaje correspondiente, el último renglón equivale al 100%.

In [58]:
%%file heapy_ex_1_2
import math

from guppy import hpy

hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
a=0
b=1
n=10**6
f=lambda x: math.exp(-x**2)
h_hat = (b-a)/n
h = hp.heap()
print(h)
print("index 1:")
print(h[1])
nodes = [a+(i+1/2)*h_hat for i in range(0,n)]
h = hp.heap()
print("After creating nodes")
print(h)    

Writing heapy_ex_1_2


In [59]:
%%bash
python3 heapy_ex_1_2

Partition of a set of 3 objects. Total size = 568 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  33      408  72       408  72 types.FrameType
     1      1  33      136  24       544  96 function
     2      1  33       24   4       568 100 float
index 1:
Partition of a set of 1 object. Total size = 136 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     1      1 100      136 100       544 400 function
After creating nodes
Partition of a set of 1000010 objects. Total size = 32699017 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1000001 100 24000024  73  24000024  73 float
     1      2   0  8697592  27  32697616 100 list
     2      2   0      577   0  32698193 100 str
     3      1   0      408   0  32698601 100 types.FrameType
     4      2   0      272   0  32698873 100 function
     5      2   0      144   0  32699017 100 builtins.weakref


In [60]:
%%file heapy_ex_1_3
import math

from guppy import hpy

hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
a=0
b=1
n=10**6
f=lambda x: math.exp(-x**2)
h_hat = (b-a)/n
h = hp.heap()
print(h)
print("index 1:")
print(h[1])
nodes = [a+(i+1/2)*h_hat for i in range(0,n)]
h = hp.heap()
print("After creating nodes")
print(h)    
sum_res = 0
for node in nodes:
    sum_res = sum_res+f(node)
h = hp.heap()
print("After sum")
print(h)

Writing heapy_ex_1_3


In [61]:
%%bash
python3 heapy_ex_1_3

Partition of a set of 3 objects. Total size = 568 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  33      408  72       408  72 types.FrameType
     1      1  33      136  24       544  96 function
     2      1  33       24   4       568 100 float
index 1:
Partition of a set of 1 object. Total size = 136 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     1      1 100      136 100       544 400 function
After creating nodes
Partition of a set of 1000010 objects. Total size = 32699017 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1000001 100 24000024  73  24000024  73 float
     1      2   0  8697592  27  32697616 100 list
     2      2   0      577   0  32698193 100 str
     3      1   0      408   0  32698601 100 types.FrameType
     4      2   0      272   0  32698873 100 function
     5      2   0      144   0  32699017 100 builtins.weakref
After sum
Partition

Utilizando *generators*:

In [62]:
%%file heapy_ex_2
import math

from guppy import hpy

hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
a=0
b=1
n=10**6
f=lambda x: math.exp(-x**2)
h_hat = (b-a)/n
h = hp.heap()
print(h)
print("index 1:")
print(h[1])
nodes = (a+(i+1/2)*h_hat for i in range(0,n))
h = hp.heap()
print("After creating nodes")
print(h)    
sum_res = 0
for node in nodes:
    sum_res = sum_res+f(node)
h = hp.heap()
print("After sum")
print(h)

Writing heapy_ex_2


In [63]:
%%bash
python3 heapy_ex_2

Partition of a set of 3 objects. Total size = 568 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  33      408  72       408  72 types.FrameType
     1      1  33      136  24       544  96 function
     2      1  33       24   4       568 100 float
index 1:
Partition of a set of 1 object. Total size = 136 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     1      1 100      136 100       544 400 function
After creating nodes
Partition of a set of 12 objects. Total size = 2145 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      2  17      832  39       832  39 types.FrameType
     1      2  17      577  27      1409  66 str
     2      2  17      272  13      1681  78 function
     3      2  17      144   7      1825  85 builtins.weakref
     4      1   8      136   6      1961  91 list
     5      1   8      112   5      2073  97 types.GeneratorType
     6      1  

Utilizando funcionalidad de *NumPy*:

In [64]:
%%file heapy_ex_3
import numpy as np

from guppy import hpy

hp = hpy()
hp.setrelheap() #Everything allocated before this call will not be in the objects you get later.
a=0
b=1
n=10**6
f_np = lambda x: np.exp(-x**2)
h_hat = (b-a)/n
h = hp.heap()
print(h)
print("index 1:")
print(h[1])
aux_vec = np.linspace(a, b, n+1)
nodes= (aux_vec[:-1]+aux_vec[1:])/2
h = hp.heap()
print("After creating nodes")
print(h)    
res = h_hat*np.sum(f_np(nodes))
print("After sum")
print(h)

Writing heapy_ex_3


In [65]:
%%bash
python3 heapy_ex_3

Partition of a set of 3 objects. Total size = 568 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1  33      408  72       408  72 types.FrameType
     1      1  33      136  24       544  96 function
     2      1  33       24   4       568 100 float
index 1:
Partition of a set of 1 object. Total size = 136 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     1      1 100      136 100       544 400 function
After creating nodes
Partition of a set of 11 objects. Total size = 8001769 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      2  18  8000208 100   8000208 100 numpy.ndarray
     1      2  18      577   0   8000785 100 str
     2      1   9      408   0   8001193 100 types.FrameType
     3      2  18      272   0   8001465 100 function
     4      2  18      144   0   8001609 100 builtins.weakref
     5      1   9      136   0   8001745 100 list
     6      1   9 

```{admonition} Ejercicio
:class: tip

Perfilar la regla de Simpson que se implementó en el primer ejercicio con `heapy`.

```

## *Basic Linear Algebra Subprograms* (BLAS)

De acuerdo a la especificación BLAS, hay diferentes niveles dependiendo del número de datos y cantidad de operaciones a realizar que involucran tales datos.

### Operación del producto interno estándar o producto punto (nivel 1 de BLAS)

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

### Ejemplo y algoritmo

In [66]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [67]:
c

-7.5

````{admonition} Comentarios

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones o [floating point operations per second](https://en.wikipedia.org/wiki/FLOPS) (flops). Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.


* Los *flops* que realiza un algoritmo es una forma de cuantificar el volumen de trabajo asociado con un cálculo. Un *flop* es una operación de punto flotante: suma, multiplicación o división. Por ejemplo, en la línea:

```python
C[i][j] = C[i][j] + A[i][k]*B[k][j]
```

se realizan $2$ *flops*. Los flops sólo representan una componente para categorizar a los algoritmos de acuerdo al trabajo que realizan, otras componentes son la transferencia o movimientos de datos, *data movement/motion*, ejecución secuencial o en paralelo y el *data locality* y *data reuse* que realizan.


* En LAPACK encontramos [sdot](http://www.netlib.org/lapack/explore-html/d0/d16/sdot_8f.html), [ddot](http://www.netlib.org/lapack/explore-html/d5/df6/ddot_8f.html) y [cdotu](http://www.netlib.org/lapack/explore-html/d7/d7b/cdotu_8f.html) para descripción de las funciones/subrutinas escritas en [Fortran](https://en.wikipedia.org/wiki/Fortran) del producto punto en los casos de precisión simple, doble o números complejos respectivamente.

````

### Operación saxpy (nivel 1 de BLAS)

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK [saxpy](http://www.netlib.org/lapack/explore-html/d8/daf/saxpy_8f.html) se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

```{admonition} Comentarios

* El símbolo "=" no se utiliza como igualdad de expresiones sino para denotar asignación (como en computación al escribir un algoritmo).

* También encontramos en LAPACK [caxpy](http://www.netlib.org/lapack/explore-html/de/da2/caxpy_8f.html) y [daxpy](http://www.netlib.org/lapack/explore-html/d9/dcd/daxpy_8f.html) para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

```

### Ejemplo y algoritmo

In [68]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [69]:
print(y)

[-4, -4, -4, -4, -4]


o en una forma *update*:

In [70]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [71]:
print(y)

[-1, 0, -5, -4, -3]


```{admonition} Comentario

La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1**, ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos de este tipo de algoritmos.

```

### Operación de multiplicación matriz-vector (nivel 2 de BLAS)

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrarla con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html) y [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) para los casos de precisión simple, doble y números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [72]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [73]:
print(A)

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]


se tiene:

### Ejemplo y algoritmo *gaxpy row oriented*

In [74]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [75]:
print(y)

[12.0, 10]


Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [76]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [77]:
print(y)

[11.0, 9]


````{admonition} Comentarios

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto $A[i,:]^Tx$

```python
for i in range(m):
    y[i]+=A[i,:]*x #producto punto
    
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. Así podemos reescribir de forma más compacta este algoritmo.

Sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado pues resulta en el error:

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-8ae2a3e9d101> in <module>
      1 for i in range(m):
----> 2     y[i]+=A[i,:]*x #producto punto
      3 

TypeError: list indices must be integers or slices, not tuple
```

a menos que incorporemos alguna paquetería que permita la **vectorización** y el uso de índices para extracción de columnas (o renglones) de $A$ como *NumPy*.

````

```{admonition} Observación
:class: tip

Obsérvese que el acceso a la matriz $A$ del algoritmo *gaxpy row oriented* es **por renglón**, de ahí el nombre.

```

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas.

### Ejemplo y algoritmo *gaxpy column oriented*

Este algoritmo ayuda a visualizar al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n A_jx_j$$

con $A_j$ la $j$-ésima columna de $A$.


In [78]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [79]:
print(y)

[12.0, 10]


````{admonition} Comentarios 

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* En el algoritmo *gaxpy column oriented* el acceso a la matriz $A$ es por columna.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el espacio generado por las columnas de $A$ y cuyas coordenadas son dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos escribir de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```python
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]
```

Sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado pues resulta en el error:

```python
TypeError                                 Traceback (most recent call last)
<ipython-input-17-93f449af9194> in <module>
      2 y=[0]*m
      3 for j in range(n):
----> 4     y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

```

a menos que incorporemos alguna paquetería que permita la **vectorización** y el uso de índices para extracción de columnas (o renglones) de $A$ como *NumPy*.

````

In [80]:
x = 2*np.ones(n)
y = np.zeros(m)

In [81]:
print(x)

[2. 2. 2. 2. 2.]


In [82]:
print(y)

[0. 0.]


In [83]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [84]:
print(A)

[[1.2 1.2 1.2 1.2 1.2]
 [1.  1.  1.  1.  1. ]]


In [85]:
for j in range(n):
    y+=A[:,j]*x[j]

In [86]:
print(y)

[12. 10.]


El algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. En *Python*:

In [87]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [88]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [89]:
print(y)

[12. 10.]


### Operación de multiplicación matriz-matriz (nivel 3 de BLAS)

El algoritmo de multiplicación de matrices $C = C + AB$ con $A \in \mathbb{R}^{m \times r}, B \in \mathbb{R}^{r \times n}, C \in \mathbb{R}^{m \times n}$ se cataloga como de nivel 3 de BLAS pues realiza una **cantidad de trabajo cúbica** sobre una **cantidad cuadrática de datos**. Ver [level 3](http://www.netlib.org/blas/#_level_3) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

En LAPACK encontramos al algoritmo de multiplicación matricial con nombres como [sgemm](http://www.netlib.org/lapack/explore-html/d4/de2/sgemm_8f.html), [dgemm](http://www.netlib.org/lapack/explore-html/d7/d2b/dgemm_8f.html) y [cgemm](http://www.netlib.org/lapack/explore-html/d6/d5b/cgemm_8f.html) para los casos de precisión simple, doble y números complejos respectivamente. 

El algoritmo de multiplicación de matrices puede escribirse en diferentes versiones. Por ejemplo la versión  **i,j,k** es:

```
for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]
    
```

y la versión **j,k,i** es:

```
for j in range(n):
    for k in range(r):
        for i in range(m):
            C[i][j]+=A[i][k]*B[k][j]
```

```{admonition} Comentarios

* Cualquiera de las versiones (hay $3!$ de ellas) involucra una cantidad de trabajo del orden $\mathcal{O}(mnr)$, la cual es cúbica. Es posible interpretar ésta cantidad de trabajo con una frase del tipo "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $8$".

* Distintas versiones tienen distinto patrón de acceso a los datos de $A, B$ y $C$. Por ejemplo, para la variante **i,j,k** el **inner loop** realiza un producto punto que requiere acceso a renglones de $A$ y columnas de $B$ en una forma izquierda-derecha y abajo-arriba respectivamente. La variante **j,k,i** involucra una operación *saxpy* en el **inner loop** y acceso por columnas a la matriz $C$ y a $A$. Un resúmen de lo anterior se presenta en el siguiente cuadro:

<img src="https://dl.dropboxusercontent.com/s/rw81crmo1b7fjvb/tabla_con_versiones_mult_matrices.png?dl=0" heigth="450" width="450">

```

### Otras versiones de la multiplicación de matrices apoyándose de las formas (i,j,k), (j,k,i),...

Si utilizamos la versión **i,j,k** podemos reescribir la multiplicación de matrices en una forma:

```
for i in range(m):
    for j in range(n)
        C[i][j]+= A[i,:].dot(B[:,j])
```

y hemos vectorizado el *inner loop* para usar productos punto que es una operación nivel 1 de BLAS.

Además *NumPy* nos provee de funcionalidad para reescribir ésta forma de productos punto en una como sigue:


```
for i in range(m):
    C[i,:]+= A[i,:]@B
```


```{admonition} Comentario

En esta reescritura se realizan $m$ operaciones *gaxpy* de nivel 2 en el *loop*. Esta versión debe ser más rápida que la que utiliza operaciones de nivel 1 de BLAS.

```

(OPERTIPICASALGLINEALYFLOPS)=

### Operaciones típicas del álgebra lineal y los *flops* que realizan

A continuación se muestra una tabla que presenta el número de *flops* realizados por distintas operaciones del álgebra lineal:

<img src="https://dl.dropboxusercontent.com/s/rvqkokicaqkwrif/tabla_con_flops_para_operaciones_alg_lineal.png?dl=0" heigth="550" width="550">


```{admonition} Comentario

Por la tabla anterior, *gaxpy* (nivel 2 de BLAS) realiza más *flops* que un producto punto. Aún así, debe cumplirse que la versión de *gaxpy* sea más rápida que la versión con productos punto (nivel 1 de BLAS) para la multiplicación de matrices pues aprovecha mejor el *data reuse* y *data locality*. Esto se logra pues *gaxpy* disminuye el número de *cache misses* y por tanto el tráfico hacia y desde el caché (*data movement/motion*).

```

### Métodos para resolver sistemas de ecuaciones lineales (SEL) o factorizaciones matriciales (FM) y los *flops* que realizan

Ver {ref}`algoritmos para resolver sistemas de ecuaciones lineales <ALGSEL>` para ejemplos de métodos directos o basados en FM e iterativos que resuelven SEL.

Los métodos iterativos no realizan FM. Hay características de la matriz (por ejemplo simetría) que no explotan en su implementación pero se benefician de otras (por ejemplo si $A$ es *sparse* o rala: muchos ceros). Otras características de tales métodos son: uso de menos memoria que los directos o basados en FM, convergen bajo condiciones simples de verificar y son sencillos de implementar.

Los métodos directos o basados en FM en general consumen más memoria que los iterativos, tienen el potencial de utilizar operaciones de nivel BLAS $3$, siempre encuentran solución si la matriz del sistema es no singular y en general requieren mejor dominio de la programación para su implementación que los iterativos. Una vez calculada la FM de la matriz $A$ se utilizan métodos para resolver SEL que resultan de la FM. Un ejemplo de tales métodos son los de sustitución hacia delante y hacia atrás para resolver SEL triangulares.

Nombremos a los métodos o algoritmos numéricos anteriores nuestros **métodos básicos del álgebra lineal para resolver SEL o FM** (MBAL para SEL o FM). Cada uno puede utilizarse para resolver SEL, FM o para otros propósitos. Por ejemplo la factorización QR también se utiliza en el cálculo de eigenvalores y eigenvectores en el algoritmo QR.

Aquí se presenta una tabla que muestra el número de flops si se desea resolver $Ax=b$ con $A \in \mathbb{R}^{n \times n}$ de algunos MBAL para SEL o FM:

<img src="https://dl.dropboxusercontent.com/s/7i01uxbxyadhuh1/flops_MBAL_ABAL.png?dl=0" heigth="250" width="250">

```{admonition} Comentarios

* Es fácil verificar que resolver $Ax = b$ calculando la inversa (por ejemplo `np.linalg.inv(A)@b`) resultan en aproximadamente $\frac{8n^3}{3}$ *flops*. 

* Los MBAL para SEL o FM sirven de apoyo para resolver SEL por **bloques**. 
```

## Blocking algorithms para multiplicación de matrices

El *data reuse* y el *data locality* se incrementa al utilizar operaciones de nivel BLAS mayores. También disminuye el *data movement/motion* hacia y desde el caché y por tanto tiene menos impacto el *bottleneck* de Von Neumann en el *performance*.

Por lo anterior hay algoritmos que se diseñan con el objetivo de aprovechar lo más posible el *data reuse* y el *data locality* y se nombran *blocking algorithms*. Entre los *blocking algorithms* encontramos a los que trabajan con matrices por bloques pues son más eficientes al utilizar un nivel $3$ de BLAS. Una matriz $A \in \mathbb{R}^{m \times n}$ por bloques se puede escribir como:

<img src="https://dl.dropboxusercontent.com/s/s77weq74yoi2rfc/matrix_A_by_blocks.png?dl=0" heigth="300" width="300">

donde: $m_1 + \dots + m_q = m$, $n_1 + \dots + n_r = n$. Con esta definición se llama a la matriz $A$, una matriz por bloques de tamaño $q \times r$.


```{admonition} Comentarios

* Hay que tener en cuenta que trabajar con matrices por bloques utiliza mayor cantidad de memoria que un algoritmo que opera a un nivel escalar esto puede ser benéfico o no dependiendo del problema (por ejemplo simetría de una matriz puede aprovecharse en este caso), máquina o arquitectura en la que se esté trabajando.

* Trabajar con *blocking algorithms* permite simplificar notación matemática. Además, su escritura ayuda a pensar en una implementación con cómputo en paralelo.

```

### Algunas operaciones en matrices por bloques

Entre las operaciones que son posibles realizar a un nivel por bloques se encuentran:

### Multiplicación por escalares

$$\begin{array}{l}
\mu \left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] = 
\left[\begin{array}{cc}
\mu A_{11} & \mu A_{12}\\
\mu A_{21} & \mu A_{22}\\
\mu A_{31} & \mu A_{32}
\end{array}
\right] 
\end{array}
$$

con $\mu \in \mathbb{R}$.

### Transposición

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right]^T = 
\left[\begin{array}{ccc}
A_{11}^T & A_{21}^T & A_{31}^T\\
A_{12}^T & A_{22}^T & A_{32}^T\\
\end{array}
\right] 
\end{array}
$$

### Suma

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] + 
\left[\begin{array}{cc}
B_{11} & B_{12}\\
B_{21} & B_{22}\\
B_{31} & B_{32}
\end{array}
\right] =
\left[\begin{array}{cc}
A_{11} + B_{11} & A_{12} + B_{12}\\
A_{21} + B_{21} & A_{22} + B_{22}\\
A_{31} + B_{31} & A_{32} + B_{32}
\end{array}
\right]
\end{array}
$$

### Multiplicación

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] \cdot
\left[\begin{array}{cc}
B_{11} & B_{12}\\
B_{21} & B_{22}\\
\end{array}
\right] =
\left[\begin{array}{cc}
A_{11}B_{11} + A_{12}B_{21} & A_{11}B_{12} + A_{12}B_{22}\\
A_{21}B_{11} + A_{22}B_{21} & A_{21}B_{12} + A_{22}B_{22}\\
A_{31}B_{11} + A_{32}B_{21} & A_{31}B_{12} + A_{32}B_{22}
\end{array}
\right]
\end{array}
$$

```{admonition} Comentario

El número de columnas de los bloques $A_{11}, A_{21}, A_{31}$ deben coincidir con el número de renglones de $B_{11}, B_{12}$. Así como con los bloques $A_{12}, A_{22}, A_{32}$ y $B_{21}, B_{22}$.

```

Considérese que se particionan a las matrices $A, B, C$ como sigue:

<img src="https://dl.dropboxusercontent.com/s/j0p89i19vf5r88k/A_B_C_matrices_by_blocks.png?dl=0" heigth="400" width="400">


Esto es, $A,B,C$ son $N \times N$ matrices por bloques con $\ell \times \ell$ bloques. Entonces, para índices $\alpha = 1,2,\dots, N$ y $\beta = 1,2,\dots,N$ se tiene que el bloque $\alpha \beta$ de $C$ se obtiene: $C_{\alpha \beta} = \displaystyle \sum_{\gamma=1}^N A_{\alpha \gamma} B_{\gamma \beta}$. 

El algoritmo una vez definido el bloque de tamaño $\ell$ es:

```python
for alpha in np.arange(N):
    i = np.arange((alpha - 1)*l + 1,alpha*l + 1) #TODO: check if index is well defined
    for beta in np.arange(N):
        j = np.arange((beta - 1)*l + 1, beta*l + 1) #TODO: check if index is well defined
        for gamma in np.arange(N):
            k = np.arange((gamma - 1)*l + 1, gamma*l + 1) #TODO: check if index is well defined
            C[i][:,j]+= A[i][:,k]*B[k][:,j]

```

```{admonition} Ejercicio
:class: tip

Implementar la multiplicación por bloques en *Python* y *R*. 

```

### Métodos por bloques para resolver sistemas de ecuaciones lineales

### Eliminación por bloque

Este método consiste en eliminar un subconjunto de variables y resolver un sistema más pequeño. Si el sistema más pequeño asociado a la submatriz puede resolverse con alguno de los MBAL para SEL o FM anteriores entonces este método puede tener más eficiencia que uno que no trabaja por bloques.


Considérese un sistema de la forma: $Ax=b$ escrito como:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right] \cdot 
\left[\begin{array}{c}
x_1\\
x_2
\end{array}
\right] =
\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]
\end{array}
$$

con $x_1 \in \mathbb{R}^{n_1}, x_2 \in \mathbb{R}^{n_2}, A_{11} \in \mathbb{R}^{n_1 \times n_1}, A_{22} \in \mathbb{R}^{n_2 \times n_2}, b_1 \in \mathbb{R}^{n_1}, b_2 \in \mathbb{R}^{n_2}$.

El método de eliminación por bloques consiste en que si $A_{11}$ es invertible, entonces se puede eliminar $x_1$ de las ecuaciones como sigue:

* De la primer ecuación por bloques: $A_{11}x_1 + A_{12}x_2 = b_1$ se obtiene: $x_1 = A_{11}^{-1}(b_1-A_{12}x_2)$.

* Sustituyendo esta relación en la segunda ecuación por bloques obtenemos la ecuación reducida: $(A_{22}-A_{21}A_{11}^{-1}A_{12})x_2 = b_2 - A_{21}A_{11}^{-1}b_1$.


```{admonition} Comentarios

* Las dos ecuaciones anteriores son equivalentes al sistema original $Ax=b$. 

* La matriz $A_{22}-A_{21}A_{11}^{-1}A_{12}$ se le nombra **complemento de Schur** del bloque $A_{11}$ en $A$ (ver [Schur complement](https://en.wikipedia.org/wiki/Schur_complement)) esto es: 

$$S = A_{22}-A_{21}A_{11}^{-1}A_{12}.$$

* Se tiene la propiedad siguiente: $S$ es no singular si y sólo si $A$ es no singular.

```

### Algoritmo: método de eliminación por bloque

> **Dados** $A$ y $A_{11}$ no singulares
>
> 1.Calcular $A_{11}^{-1}A_{12}$ y $A_{11}^{-1}b_1$ teniendo cuidado en **no** calcular la inversa sino un sistema de ecuaciones lineales:
>
>>Para realizar la multiplicación $A_{11}^{-1}b_1$ definimos $y=A_{11}^{-1}b_1$ y por tanto $A_{11}y = b_1$ ($A_{11}$ es no singular). Así, resolvemos para $y$ el sistema anterior y habremos calculado $A_{11}^{-1}b_1$. Similarmente definimos $Y=A_{11}^{-1}A_{12}$ con lo que se tiene $A_{11}Y=A_{12}$. Resolvemos para $Y \in \mathbb{R} ^{n_1 \times n_1}$ y habremos calculado $A_{11}^{-1}A_{12}$.
>
>2.Calcular el complemento de Schur del bloque $A_{11}$ en $A$: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. Calcular $ \hat{b} = b_2-A_{21}A_{11}^{-1}b_1$.
>
>3.Resolver $Sx_2 = \hat{b}$.
>
>4.Resolver $A_{11}x_1 = b_1-A_{12}x_2$.


```{admonition} Comentario

En los pasos que se requieren resolver sistemas de ecuaciones lineales se utilizan los MBAL para SEL o FM.

```

### *Factor-Solve method*

El método de factor solve consiste en expresar a la matriz del sistema $Ax = b$ como un producto de la forma: $A = A_1A_2 \cdots A_k$. Si $A$ es no singular entonces: $x = A^{-1}b = A_k^{-1}A_{k-1}^{-1} \cdots A_1^{-1}b$.

De esta forma, $x$ puede calcularse de "derecha a izquierda":

### Descripción matemática del algoritmo

$$
\begin{eqnarray}
z_1&:=&A_1^{-1}b \nonumber \\
z_2&:=&A_2^{-1}z_1 &= A_2^{-1}A_1^{-1}b \nonumber\\
\vdots \nonumber\\
z_{k-1}&:=&A_{k-1}^{-1}z_{k-2} &= A_{k-1}^{-1} \cdots A_1^{-1}b \nonumber\\
x &:=&A_k^{-1}z_{k-1} &= A_k^{-1} \cdot A_{k-1}^{-1} \cdots A_1^{-1}b \nonumber
\end{eqnarray}
$$

```{admonition} Comentarios

* El paso que consiste en factorizar a la matriz $A$ es el paso nombrado **factor** y el usado en resolver el sistema $A_iz_i = z_{i-1}$ es nombrado **solve**.

* Normalmente $A_i$ tiene una estructura aprovechable (por ejemplo triangular superior/inferior) para resolver el sistema $A_iz_i = z_{i-1}$. Por ejemplo en el caso de la factorización $LU$ de $A$ se cumple $PA = LU$ con $P$ matriz de permutación, $L$ triangular inferior con $1$'s en la diagonal y $U$ triangular superior. Entonces:

    * Paso 1: encontrar factores $P,L,U$ tales que $PA=LU$.
    
    * Paso 2: resolver $Ld=Pb$.
    
    * Paso 3: resolver $Ux=d$.
    
```

### Método de eliminación por bloques escrito como un *factor-solve method*:

El método de *block elimination* puede escribirse como un *factor-solve method*:

Paso $1$:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right]=
\left[\begin{array}{cc}
A_{11} & 0 \\
A_{21} & S
\end{array}
\right]
\left[\begin{array}{cc}
I & A_{11}^{-1}A_{12} \\
0 & I
\end{array}
\right]
\end{array}
$$



Paso $2$. Resolver por el método de sustitución hacia delante por bloques:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & 0\\
A_{21} & S
\end{array}
\right] \cdot
\left[\begin{array}{c}
z_1 \\
z_2
\end{array}
\right] = 
\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]
\end{array}
$$

Paso $3$. Resolver por el método de sustitución hacia atrás por bloques:

$$\begin{array}{l}
\left[ \begin{array}{cc}
I & A_{11}^{-1}A_{12}\\
0 & I
\end{array}
\right] \cdot
\left[\begin{array}{c}
x_1 \\
x_2
\end{array}
\right] = 
\left[\begin{array}{c}
z_1\\
z_2
\end{array}
\right]
\end{array}
$$

```{admonition} Comentario

En los pasos que se requieren resolver sistemas de ecuaciones lineales se utilizan los MBAL para SEL o FM.


```

### OpenBLAS y *NumPy*

En esta sección se sugiere para reproducibilidad de resultados utilizar la *AMI* y la instancia de AWS ambas descritas al inicio y la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` de la región `us-east-1` (Virginia) en el mismo tipo de instancia. Tal AMI se construyó a partir de una *AMI* `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)


````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=no-OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

### Resultados utilizando `opt2-aws-educate-openblas-04-04-2021`

Verificamos que *NumPy* está usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

In [10]:
print(np.show_config())

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
None


### Multiplicación de matrices

Hacemos una prueba de medición de tiempos:

In [2]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)

In [3]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [4]:
%timeit -n 1 -r 7 A@B

3.02 s ± 67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Factorización LU

### Resultados utilizando `opt2-aws-educate-no-openblas-04-04-2021`

Verificamos que *NumPy* **no** está usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)

In [2]:
print(np.show_config())

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
None


### Multiplicación de matrices

Hacemos una prueba de medición de tiempos:

In [3]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)

In [4]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [5]:
%timeit -n 1 -r 7 A@B

3.22 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```{admonition} Comentarios

* La implementación de la API de BLAS con OpenBLAS que se instala mediante `sudo apt-get install -y libopenblas-dev` y *NumPy* instalado vía `pip install numpy --no-binary numpy` resulta para el algoritmo de multiplicación de matrices ser un **poco más rápido** que con la implementación de la API de BLAS integrada en la instalación de *NumPy* vía `pip install numpy`.

* Más *cores* reducen el tiempo de cómputo, para ver esto probar lo anterior con máquinas con menos *cores*.

```

### OpenBLAS y librerías de cómputo matricial de *R*

En esta sección se sugiere para reproducibilidad de resultados utilizar la *AMI* y la instancia de AWS ambas descritas al inicio y la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` de la región `us-east-1` (Virginia) en el mismo tipo de instancia. Tal AMI se construyó a partir de una *AMI* `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)


````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=no-OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

### Resultados utilizando `opt2-aws-educate-openblas-04-04-2021`

Verificamos que las librerías de cómputo matricial de *R* están usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

In [1]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_0.4.2         digest_0.6.27       utf8_1.2.1         
 [4] crayon_1.4.1        IRdisplay_1.0       repr_1.1.3         
 [7] lifecycle_1.0.0     jsonlite_1.7.2      evaluate_0.14      
[10] pillar_1.5.1        rlang_0.4.10        uuid_0.1-4         
[13] ellipsis_0.3.1      IRkernel_

In [2]:
set.seed(2020)
m<-10**4
r<-10**4
A <- matrix(runif(m*r), m, r)

In [3]:
set.seed(2021)
r<-10**4
n<-10**4
B <- matrix(runif(r*n), r, n)

In [5]:
library(microbenchmark)

```{admonition} Comentario

El paquete de [microbenchmark](https://www.rdocumentation.org/packages/microbenchmark/versions/1.4-7/topics/microbenchmark) nos ayuda a la medición de estadísticas para pequeñas piezas de código.

```

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [10]:
mbk<-microbenchmark(
    A%*%B,
    times=7
    )

In [11]:
print(mbk)

Unit: seconds
    expr      min       lq     mean   median       uq      max neval
 A %*% B 2.600818 2.723897 2.952135 2.833146 3.038394 3.706399     7


### Resultados utilizando `opt2-aws-educate-no-openblas-04-04-2021`

Verificamos que las librerías de cómputo matricial de *R* **no** están usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)

In [1]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.9.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.9.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_0.4.2         digest_0.6.27       utf8_1.2.1         
 [4] crayon_1.4.1        IRdisplay_1.0       repr_1.1.3         
 [7] lifecycle_1.0.0     jsonlite_1.7.2      evaluate_0.14      
[10] pillar_1.5.1        rlang_0.4.10        uuid_0.1-4         
[13] ellipsis_0.3.1      IRkernel_1.1.1.9000 too

In [2]:
set.seed(2020)
m<-10**4
r<-10**4
A <- matrix(runif(m*r), m, r)

In [3]:
set.seed(2021)
r<-10**4
n<-10**4
B <- matrix(runif(r*n), r, n)

In [4]:
library(microbenchmark)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [ ]:
mbk<-microbenchmark(
    A%*%B,
    times=1 #just one time as it takes too much time
    )

In [ ]:
print(mbk) #after ten minutes didnt finish

````{admonition} Comentarios

* La implementación de la API de BLAS con OpenBLAS que se instala mediante `sudo apt-get install -y libopenblas-dev` y las librerías de cómputo matricial de *R* que utilizan tal instalación resulta para el algoritmo de multiplicación de matrices ser mucho más rápido que con la implementación de la API de BLAS integrada en la instalación de *R*.

* Más *cores* reducen el tiempo de cómputo, para ver esto probar lo anterior con máquinas con menos *cores*.

* En *R* se puede cambiar de implementación de la API de BLAS con el comando: 

```sudo update-alternatives --config libblas.so.3-x86_64-linux-gnu```


````

## Perfilamiento: medición de tiempo en el sistema operativo Ubuntu 20.04

### [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html)

Ver también: [Wikipedia: Time_Unix](https://en.wikipedia.org/wiki/Time_(Unix))


In [20]:
%%file Rcf.py
import math

from pytest import approx
from scipy.integrate import quad
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res
if __name__=="__main__": #añadimos este bloque para ejecución de la función Rcf
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))

Writing Rcf.py


In [19]:
%%bash
/usr/bin/time -p python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


real 0.84
user 2.57
sys 2.20


In [20]:
%%bash
/usr/bin/time -v python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


	Command being timed: "python3 Rcf.py"
	User time (seconds): 2.55
	System time (seconds): 2.34
	Percent of CPU this job got: 612%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.79
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 97688
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 20846
	Voluntary context switches: 87
	Involuntary context switches: 877104
	Swaps: 0
	File system inputs: 0
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


```{admonition} Comentario

Una breve explicación del *output* se puede encontrar [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html). Por ejemplo, para el caso de `Major (requiring I/O)` nos interesa que sea $0$ pues indica que el sistema operativo tiene que cargar páginas de datos del disco pues tales datos ya no residen en RAM (por alguna razón).

```

```{admonition} Observación
:class: tip

La medición `real` que regresa `/usr/bin/time` es el equivalente al `Wall time` de `%time`.


```

## Perfilamiento: medición de uso de CPU en el sistema operativo GNU/Linux

### [perf](https://github.com/torvalds/linux/tree/master/tools/perf)

En GNU/Linux existe la herramienta `perf` que nos ayuda a calcular métricas de desempeño de la CPU. Los siguientes ejemplos se ejecutaron con la AMI `opt2-aws-educate-openblas-04-04-2021` y la instancia de AWS, ambas descritas al inicio.

### Ejemplo de cálculo de la norma $2$ al cuadrado de un vector

In [1]:
%%file norm_square.py
n=10**7
vector=list(range(n))
norm=0
for v in vector:
    norm+=v*v

Writing norm_square.py


```{margin}

Esta línea de `echo` es importante para permitir que *user* `ubuntu` ejecute `perf` sin `sudo`.

```

In [2]:
%%bash
echo "-1" |sudo tee -a /proc/sys/kernel/perf_event_paranoid

-1


In [3]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

        5123821017      cycles                                                        ( +-  0.55% )  (74.95%)
       14073511390      instructions              #    2.75  insn per cycle           ( +-  0.69% )  (75.01%)
          42426825      cache-references                                              ( +-  0.20% )  (75.06%)
          19997224      cache-misses              #   47.133 % of all cache refs      ( +-  0.13% )  (74.98%)

            1.9227 +- 0.0113 seconds time elapsed  ( +-  0.59% )



```{admonition} Comentarios

* Con `perf` se repiten las mediciones utilizando la *flag* `-r`. Con la *flag* `-e` se enlistan las métricas a calcular. `-S` llama a [sync](http://man7.org/linux/man-pages/man2/sync.2.html) antes de iniciar la ejecución del programa. `-A` no agregar los conteos a lo largo de los *cores* monitoreados. Ver `perf stat --help`.

* En el ejemplo anterior además de los ciclos e instrucciones se calculan los *cache references* y los *cache misses*.  Esencialmente el sistema de memoria buscó el número que aparece en *cache-references* por datos o instrucciones en el caché y de éstas búsquedas, en *cache-misses*, se repora el número (y porcentaje) de búsquedas fallidas que no estuvieron en memoria. 

* Recuérdese que la fragmentación de datos incrementa el número de *cache misses*.

* Se reporta el IPC al lado de *instructions* con nombre *insn per cycle*. Un alto IPC indica una alta transferencia de instrucciones de la unidad de memoria hacia la unidad de cómputo y un menor IPC indica más **stall cycles**, ver [pipeline stall](https://en.wikipedia.org/wiki/Pipeline_stall). Se debe evaluar si *a high rate of instructions* indica un *high rate of actual work completed* (p.ej. un *loop* tiene una alta tasa de IPC pero no siempre se realiza trabajo útil). Ver sección *CPU statistics* en la [liga](http://www.brendangregg.com/perf.html). `perf` también tiene una métrica para medir el número de ciclos que se utilizaron para esperar a ejecutar instrucciones (los cores están *stalled*). Las métricas son: `stalled-cycles-frontend` y `stalled-cycles-backend`.

```

También podemos obtener las estadísticas por core. Obsérvese que la salida anterior de `perf` calculó el IPC a partir del número de instrucciones y ciclos ahí reportados. El mismo cálculo se realiza a continuación:

In [8]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

CPU0               298120352      cycles                                                        (75.11%)
CPU1               301648607      cycles                                                        (75.03%)
CPU2               310307744      cycles                                                        (75.03%)
CPU3               309422661      cycles                                                        (75.05%)
CPU4               290664122      cycles                                                        (75.11%)
CPU5               306978407      cycles                                                        (75.05%)
CPU6               300076931      cycles                                                        (75.07%)
CPU7               303777036      cycles                                                        (75.09%)
CPU8               314116473      cycles                                                        (75.11

Otras métricas pueden ser obtenidas si ejecutamos `perf` sólo con la *flag* `-r`:

In [9]:
%%bash
perf stat -S -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

           1927.51 msec task-clock                #    1.000 CPUs utilized            ( +-  0.62% )
                 1      context-switches          #    0.000 K/sec                    ( +- 20.20% )
                 0      cpu-migrations            #    0.000 K/sec                  
             99924      page-faults               #    0.052 M/sec                    ( +-  0.00% )
        5150266707      cycles                    #    2.672 GHz                      ( +-  0.58% )  (74.97%)
       14131213509      instructions              #    2.74  insn per cycle           ( +-  0.74% )  (75.01%)
        2929512564      branches                  # 1519.839 M/sec                    ( +-  0.85% )  (75.04%)
           1173976      branch-misses             #    0.04% of all branches          ( +-  0.68% )  (74.98%)

            1.9279 +- 0.0119 seconds time elapsed  ( +-  0.62% )



```{admonition} Comentarios

* La métrica de *task-clock* indica cuántos *clock cycles* tomó nuestra tarea y se reporta en milisegundos. Obsérvese que también se indica cuántos CPU's fueron utilizados. En el *output* anterior puede obtenerse no exactamente igual a 1 para `CPUs utilized` pues el programa no sólo involucra trabajo de CPU sino también de alojamiento de memoria. 

* *context-switches* y *cpu-migrations* indican cuánto se detuvo el programa para realizar:

    * operaciones relacionadas con el kernel del sistema (por ejemplo I/O).
    * ejecución de otras aplicaciones.
    * alojamiento de la ejecución en un core distinto (lo que ocasiona que haya movimiento de datos o instrucciones hacia el caché de otro core).
    
la idea es que nuestro programa tenga un número pequeño de éstas métricas.

* *page-faults* se relaciona con el alojamiento de memoria a partir del kernel del sistema operativo. Ocasiona que se detenga la ejecución del programa y por tanto deseamos que ésta métrica no sea muy grande. Ver [Page fault](https://en.wikipedia.org/wiki/Page_fault).

* Para el *branching* interesa que el número de *branch misses* sea pequeño.


```

Podemos generar la salida anterior por *core*:

In [10]:
%%bash
perf stat -S --all-cpus -A -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                 2004.52 msec cpu-clock                 #    1.001 CPUs utilized          
CPU1                 2004.52 msec cpu-clock                 #    1.001 CPUs utilized          
CPU2                 2004.96 msec cpu-clock                 #    1.001 CPUs utilized          
CPU3                 2004.99 msec cpu-clock                 #    1.001 CPUs utilized          
CPU4                 2004.97 msec cpu-clock                 #    1.001 CPUs utilized          
CPU5                 2004.94 msec cpu-clock                 #    1.001 CPUs utilized          
CPU6                 2004.91 msec cpu-clock                 #    1.001 CPUs utilized          
CPU7                 2004.88 msec cpu-clock                 #    1.001 CPUs utilized          
CPU8                 2004.85 msec cpu-clock                 #    1.001 CPUs utilized          
CPU9                 2004.81 msec cpu-clock                 #    1.001 CPUs utilized   

Para contrastar las salidas anteriores con un mejor uso del caché y de los *cores* se utiliza a continuación el paquete de `numpy`:

In [3]:
%%file norm_square_numpy.py
import numpy as np
n=10**7
vector=np.arange(n)
vector.dot(vector)

Writing norm_square_numpy.py


In [13]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

       10883030366      cycles                                                        ( +-  1.92% )  (74.13%)
        3317792148      instructions              #    0.30  insn per cycle           ( +-  2.36% )  (75.34%)
          20753464      cache-references                                              ( +-  1.15% )  (75.80%)
           1743193      cache-misses              #    8.400 % of all cache refs      ( +-  7.94% )  (74.73%)

           0.23934 +- 0.00512 seconds time elapsed  ( +-  2.14% )



```{admonition} Comentarios

* Obsérvese que es **más rápido** este programa para el número $n$ que se está utilizando en comparación con el programa anterior sin vectorizar.

* El número de instrucciones y el número de *cache-misses* son **menores** al reportado con el programa anterior sin vectorizar.

```

Métricas por core:

In [14]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                62508379      cycles                                                        (76.32%)
CPU1                26857115      cycles                                                        (76.26%)
CPU2                27978652      cycles                                                        (76.29%)
CPU3                24328931      cycles                                                        (76.22%)
CPU4                25400584      cycles                                                        (76.22%)
CPU5                23141653      cycles                                                        (76.12%)
CPU6                24341914      cycles                                                        (76.16%)
CPU7                22866352      cycles                                                        (76.25%)
CPU8                26054803      cycles                                                        (76.35

```{admonition} Comentarios

* Se observa que la métrica IPC está más uniforme a través de los cores lo que indica un mejor *load balancing*.

* Además, el $\%$ de *cache-misses* es menor que en el caso no vectorizado.

```

También podemos obtener las salidas anteriores de `perf` al usar `numpy`: 

In [16]:
%%bash
perf stat -S -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

           4009.45 msec task-clock                #   17.208 CPUs utilized            ( +-  1.33% )
            564444      context-switches          #    0.141 M/sec                    ( +-  5.07% )
                50      cpu-migrations            #    0.013 K/sec                    ( +-  6.76% )
              4686      page-faults               #    0.001 M/sec                    ( +-  0.75% )
       10567628399      cycles                    #    2.636 GHz                      ( +-  1.28% )  (73.97%)
        3602865118      instructions              #    0.34  insn per cycle           ( +-  1.75% )  (75.34%)
         763664359      branches                  #  190.466 M/sec                    ( +-  1.77% )  (75.77%)
          15063370      branch-misses             #    1.97% of all branches          ( +-  1.82% )  (74.93%)

           0.23300 +- 0.00349 seconds time elapsed  ( +-  1.50% )



In [18]:
%%bash
perf stat -S --all-cpus -A -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                  233.86 msec cpu-clock                 #    0.883 CPUs utilized          
CPU1                  233.84 msec cpu-clock                 #    0.883 CPUs utilized          
CPU2                  233.83 msec cpu-clock                 #    0.883 CPUs utilized          
CPU3                  233.85 msec cpu-clock                 #    0.883 CPUs utilized          
CPU4                  233.86 msec cpu-clock                 #    0.883 CPUs utilized          
CPU5                  233.87 msec cpu-clock                 #    0.883 CPUs utilized          
CPU6                  233.89 msec cpu-clock                 #    0.883 CPUs utilized          
CPU7                  233.87 msec cpu-clock                 #    0.883 CPUs utilized          
CPU8                  233.88 msec cpu-clock                 #    0.883 CPUs utilized          
CPU9                  233.89 msec cpu-clock                 #    0.883 CPUs utilized   

### perf OpenBLAS: *gaxpy row, column oriented* (nivel 2)

Utilizaremos `perf` en lo que sigue para evaluar las métricas de IPC, *cache-references*, *cache-misses* y medir el tiempo de ejecución. Además, esto permitirá comparar los tiempos de ejecución del algoritmo *gaxpy* (nivel BLAS 2), con vectorización y sin vectorización.

In [102]:
np.random.seed(2020)
m=10**4
n=10**4
A=np.random.rand(m,n)
file='A.txt'
np.savetxt(file,A)

Utilizando listas para construir a la matriz y a los vectores:

In [103]:
%%file mult_matrix_vector.py
m=10**4
n=10**4
x=[2.5]*n
y=[0]*m
A = []
file='A.txt'
with open(file,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])      
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Writing mult_matrix_vector.py


### *gaxpy column oriented* caso no vectorizado

In [104]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (7 runs):

      267299273562      cycles                                                        ( +-  0.43% )  (75.00%)
      593616793103      instructions              #    2.22  insn per cycle           ( +-  0.44% )  (75.00%)
        2617234548      cache-references                                              ( +-  0.26% )  (75.00%)
         426473660      cache-misses              #   16.295 % of all cache refs      ( +-  0.78% )  (75.00%)

            99.822 +- 0.532 seconds time elapsed  ( +-  0.53% )



Métricas por *core*:

In [105]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector.py


 Performance counter stats for 'system wide' (7 runs):

CPU0              3546811536      cycles                                                        (75.00%)
CPU1              3532817847      cycles                                                        (75.00%)
CPU2              3537667402      cycles                                                        (75.00%)
CPU3              3579132369      cycles                                                        (75.00%)
CPU4              3561009406      cycles                                                        (75.00%)
CPU5              3547701460      cycles                                                        (75.00%)
CPU6              3523392168      cycles                                                        (75.00%)
CPU7              3527550856      cycles                                                        (75.00%)
CPU8              3581413600      cycles                                                        (75.00%

### *gaxpy column oriented* caso vectorizando

Vectorizando con *NumPy*:

In [106]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**4
n=10**4
x = 2.5*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Writing mult_matrix_vector_numpy.py


In [107]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (7 runs):

      256662320683      cycles                                                        ( +-  0.68% )  (74.96%)
      588618832448      instructions              #    2.29  insn per cycle           ( +-  0.05% )  (74.99%)
        2373542969      cache-references                                              ( +-  0.21% )  (75.03%)
         358718803      cache-misses              #   15.113 % of all cache refs      ( +-  0.31% )  (75.02%)

            91.933 +- 0.639 seconds time elapsed  ( +-  0.70% )



Métricas por *core*:

In [108]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'system wide' (7 runs):

CPU0              3517195213      cycles                                                        (75.00%)
CPU1              3623926958      cycles                                                        (75.00%)
CPU2              6529863873      cycles                                                        (75.00%)
CPU3              3675939897      cycles                                                        (75.00%)
CPU4              3634674054      cycles                                                        (75.00%)
CPU5              3607564802      cycles                                                        (75.00%)
CPU6              3640545147      cycles                                                        (75.00%)
CPU7              3604545197      cycles                                                        (75.00%)
CPU8              3665767111      cycles                                                        (75.00%

### *gaxpy row oriented* caso vectorizando

In [109]:
%%file mult_matrix_vector_numpy_row_oriented.py
import numpy as np
m=10**4
n=10**4
x = 2.5*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for i in np.arange(m):
    y[i]+=A[i,:].dot(x)

Writing mult_matrix_vector_numpy_row_oriented.py


In [110]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector_numpy_row_oriented.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy_row_oriented.py' (7 runs):

      253356256063      cycles                                                        ( +-  0.22% )  (74.94%)
      587323124427      instructions              #    2.32  insn per cycle           ( +-  0.02% )  (75.01%)
        2245627265      cache-references                                              ( +-  0.19% )  (75.04%)
         358281499      cache-misses              #   15.955 % of all cache refs      ( +-  0.47% )  (75.02%)

            90.230 +- 0.251 seconds time elapsed  ( +-  0.28% )



In [111]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_vector_numpy_row_oriented.py


 Performance counter stats for 'system wide' (7 runs):

CPU0              3494857184      cycles                                                        (75.00%)
CPU1              3620694160      cycles                                                        (75.00%)
CPU2              3485877647      cycles                                                        (75.00%)
CPU3              3666323043      cycles                                                        (75.00%)
CPU4              3602130097      cycles                                                        (75.00%)
CPU5              3594166927      cycles                                                        (75.00%)
CPU6              3595914756      cycles                                                        (75.00%)
CPU7              3601064499      cycles                                                        (75.00%)
CPU8              3663235633      cycles                                                        (75.00%

```{admonition} Comentarios

* No se observa una gran diferencia en tiempo de ejecución entre el caso vectorizado y no vectorizado en los niveles 1 y 2 de BLAS salvo que en el caso vectorizando se tiene un $\%$ mayor de *cache-misses*. 

* Otra opción que se tiene es utilizar la siguiente implementación ya que `numpy` soporta operaciones del tipo matriz-vector sin necesidad de utilizar un *for loop*.

```

### perf OpenBLAS: nivel 3 multiplicación de matrices

In [90]:
%%file mult_matrix_matrix_numpy_openblas.py
import numpy as np
m=10**4
r=10**4
n=10**4

np.random.seed(2020)
m=10**4
r=10**4
A=np.random.rand(m,r)

np.random.seed(2021)
r=10**4
n=10**4
B=np.random.rand(r,n)

A@B

Writing mult_matrix_matrix_numpy_openblas.py


In [93]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_matrix_numpy_openblas.py


 Performance counter stats for 'system wide' (7 runs):

CPU0              7133791024      cycles                                                        (75.03%)
CPU1              7156292470      cycles                                                        (75.03%)
CPU2              7220608589      cycles                                                        (75.04%)
CPU3              7189283352      cycles                                                        (75.02%)
CPU4              7264398959      cycles                                                        (75.02%)
CPU5              7113528745      cycles                                                        (75.03%)
CPU6              7203442255      cycles                                                        (75.01%)
CPU7              7103683132      cycles                                                        (75.02%)
CPU8              7198249150      cycles                                                        (75.02%

### Comparación de versiones de multiplicación de matrices utilizando nivel 1 vs nivel 2 de BLAS

Utilizamos matrices $A, B \in \mathbb{R}^{10^3 \times 10^3}$ pseudoaleatorias:

In [94]:
np.random.seed(2020)
m=10**3
r=10**3

A=np.random.rand(m,r)
fileA_10_3='A_10_3.txt'
np.savetxt(fileA_10_3,A)

In [95]:
np.random.seed(2021)
m=10**3
r=10**3

B=np.random.rand(m,r)
fileB_10_3='B_10_3.txt'
np.savetxt(fileB_10_3,B)

In [96]:
%%file mult_matrix_matrix_numpy_dot_product.py
import numpy as np
m=10**3
n=10**3

fileA_10_3='A_10_3.txt'
fileB_10_3='B_10_3.txt'
A = np.loadtxt(fileA_10_3)
B = np.loadtxt(fileB_10_3)
C = np.zeros((m,n))
for i in np.arange(m):
        for j in np.arange(n):
                C[i][j]+= A[i,:].dot(B[:,j])

Writing mult_matrix_matrix_numpy_dot_product.py


In [98]:
%%bash
perf stat -S --all-cpus -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (7 runs):

       36944844215      cycles                                                        ( +-  2.02% )  (75.05%)
       34136304454      instructions              #    0.92  insn per cycle           ( +-  0.63% )  (75.05%)
         338823740      cache-references                                              ( +-  0.44% )  (75.05%)
           4700721      cache-misses              #    1.387 % of all cache refs      ( +-  3.03% )  (75.02%)

            5.5781 +- 0.0290 seconds time elapsed  ( +-  0.52% )



In [97]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (7 runs):

CPU0               510519119      cycles                                                        (75.04%)
CPU1               519702645      cycles                                                        (75.04%)
CPU2               520533947      cycles                                                        (75.04%)
CPU3               521501814      cycles                                                        (75.05%)
CPU4               545961014      cycles                                                        (75.02%)
CPU5             14207577303      cycles                                                        (75.04%)
CPU6               492710418      cycles                                                        (75.04%)
CPU7               488038757      cycles                                                        (75.05%)
CPU8               484330139      cycles                                                        (75.06%

In [99]:
%%file mult_matrix_matrix_numpy_dot_product_gaxpy.py
import numpy as np
m=10**3
n=10**3
fileA_10_3='A_10_3.txt'
fileB_10_3='B_10_3.txt'
A = np.loadtxt(fileA_10_3)
B = np.loadtxt(fileB_10_3)
C = np.zeros((m,n))
for i in np.arange(m):
    C[i,:] = A[i,:]@B

Writing mult_matrix_matrix_numpy_dot_product_gaxpy.py


In [100]:
%%bash
perf stat -S --all-cpus -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product_gaxpy.py


 Performance counter stats for 'system wide' (5 runs):

       66773367195      cycles                                                        ( +- 14.72% )  (75.08%)
       29948977121      instructions              #    0.45  insn per cycle           ( +-  8.52% )  (75.08%)
         381917316      cache-references                                              ( +-  2.56% )  (75.13%)
          16449359      cache-misses              #    4.307 % of all cache refs      ( +- 21.72% )  (75.09%)

            2.4576 +- 0.0743 seconds time elapsed  ( +-  3.02% )



In [101]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product_gaxpy.py


 Performance counter stats for 'system wide' (5 runs):

CPU0              1743710495      cycles                                                        (75.07%)
CPU1              1461889304      cycles                                                        (75.06%)
CPU2              1647203296      cycles                                                        (75.05%)
CPU3              1422679558      cycles                                                        (75.05%)
CPU4              1503569453      cycles                                                        (75.06%)
CPU5              1310469979      cycles                                                        (75.04%)
CPU6              1492379372      cycles                                                        (75.05%)
CPU7              1458187701      cycles                                                        (75.06%)
CPU8              1503827496      cycles                                                        (75.06%

```{admonition} Comentario

Se cumple lo mencionado en {ref}`operaciones típicas del álgebra lineal y los *flops* que realizan <OPERTIPICASALGLINEALYFLOPS>`.

```

```{admonition} Ejercicio
:class: tip

Perfilar la implementación de multiplicación por bloques en *Python* y *R* con `perf`.

```

## Otras herramientas para perfilamiento

* [pmc-cloud-tools](https://github.com/brendangregg/pmc-cloud-tools) que ayuda a la medición de uso de CPU en el sistema operativo GNU/Linux.

* Una de las dependencias de *line_profiler* es [psutil](https://pypi.org/project/psutil/) que ayuda al monitoreo y manejo de procesos y monitoreo del sistema computacional.

* Herramientas para monitoreo de memoria en *Python*: 

    * [filprofiler](https://github.com/pythonspeed/filprofiler)
    
    * [tracemalloc](https://docs.python.org/3/library/tracemalloc.html)
    
    * [pympler](https://github.com/pympler/pympler) que contiene [muppy](https://pympler.readthedocs.io/en/latest/muppy.html#muppy) para identificar *memory leaks*.
    
    
* [resource](https://docs.python.org/3/library/resource.html) para medición y control de recursos del sistema utilizados por un programa en *Python*.



```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

3. G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University Press, 2013.